In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [2]:
# dataset = pd.read_csv('validation_data_prediction.csv')
dataset = pd.read_csv('testing_data_prediction.csv')
leverage_factor = 10
increase_factor = 1.5

In [3]:
dataset["new_rfr"] = dataset["daily_risk_free_rate"] * increase_factor

In [4]:
def first_trading_strategy(predictions: pd.DataFrame,
                           initial_cash: float = 200000,
                           leverage_factor: float = leverage_factor):
    
    # initialize the variables
    cash = [initial_cash]
    cash_high = [initial_cash]
    
    margin = [initial_cash]
    margin_high = [initial_cash]
    
    position = [0]
    re = [0]
    
    bank_account = [0]
    high_bank_account = [0]
    
    previous_price = float('nan')
    count = 0
    # iterate through the predictions
    for _, row in predictions[:-1].iterrows():
        current_date = row['Date']
        current_price = row['Close']
        
        current_rfr = row['daily_risk_free_rate']
        current_high_rfr = row['new_rfr']
        
        tomorrow_prediction = row['Tomorrow Prediction']
        
        current_cash_high = cash_high[-1]
        current_cash = cash[-1]
        
        current_margin = margin[-1]
        current_margin_high = margin_high[-1]
        
        current_position = position[-1]
        current_re = re[-1]
        current_bank_account = bank_account[-1]
        currnet_high_bank_account = high_bank_account[-1]
        # checking the excess returns
        # excess return happens when the price goes down and we have a short position
        # or the price goes up and we have a long position     
        if abs(current_position * current_price) > current_margin * leverage_factor:
            if current_position > 0:
                excess_return = (current_position * current_price) - (current_margin * leverage_factor)
                excess_stock = excess_return / current_price
                current_position -= excess_stock
                current_bank_account += excess_return
                
            elif current_position < 0:
                excess_return = -current_position * current_price - current_margin * leverage_factor
                excess_stock = excess_return / current_price
                current_position += excess_stock
                
        if current_position > 0:
            current_re = (current_price - previous_price) / previous_price
            today_profits = (current_position * previous_price) * (current_re - current_rfr)
    
            today_profits_highrfr = (current_position * current_price) * (current_re - current_high_rfr)
            current_margin += today_profits
            current_margin_high += today_profits_highrfr
        
        if current_position < 0:
            
            current_re = (previous_price - current_price) / previous_price
            
            today_profits = (current_position * previous_price) * (current_re + current_rfr)
            today_profits_highrfr = (current_position * previous_price) * (current_re + current_high_rfr)
            
            current_margin += today_profits
            current_margin_high += today_profits_highrfr
            
        # check if we should open a position
        # if we think the price will go up, we will close the short position and 
        # open a long position
        if tomorrow_prediction > current_price:
            # if we have a short position, we will close it
            if current_position < 0:
                current_position = 0
                
            # don't open a long position if we already have one
            if current_position == 0:
                stock_for_long = (current_margin * leverage_factor) / current_price
                current_position += stock_for_long
    
        # check if we should close a position
        # if we think the price will go down, we will close the long position 
        # but not open a short position
        elif tomorrow_prediction < current_price:
            # if we have a long position, we will close it
            if current_position > 0:
                current_position = 0
                    
            if current_position == 0:               
                stock_for_short = (current_margin * leverage_factor ) / current_price
                current_position -= stock_for_short
                short_price = current_price

        if current_margin < 0:
            print(f"Game Over! You lost all your money.")
            return cash, margin, position, re, bank_account, price, high_bank_account
                
        previous_price = current_price
        
        currnet_high_bank_account = current_bank_account
        
        current_interest = current_bank_account * current_rfr
        current_bank_account += current_interest
        
        current_high_interest = currnet_high_bank_account * current_high_rfr
        currnet_high_bank_account += current_high_interest
        
        current_cash = current_bank_account
        current_cash += current_margin
        
        current_cash_high = currnet_high_bank_account
        current_cash_high += current_margin_high
        
        # update the variables
        cash.append(current_cash)
        cash_high.append(current_cash_high)
        margin.append(current_margin)
        position.append(current_position)
        re.append(current_re)
        bank_account.append(current_bank_account)
        high_bank_account.append(currnet_high_bank_account)
        margin_high.append(current_margin_high)
        count += 1

    current_price = predictions.iloc[-1]['Close']
    current_date = predictions.iloc[-1]['Date']
    current_cash = cash[-1]
    current_cash_high = cash_high[-1]
    current_margin = margin[-1]
    current_position = position[-1]
    current_re = re[-1]
    current_bank_account = bank_account[-1]
    currnet_high_bank_account = high_bank_account[-1]
    current_margin_high = margin_high[-1]
    
    current_interest = current_bank_account * current_rfr
    current_bank_account += current_interest
    
    current_high_interest = currnet_high_bank_account * current_high_rfr
    currnet_high_bank_account += current_high_interest
    
    if current_position > 0:
        current_re = (current_price - previous_price) / previous_price
        
        today_profits = (current_position * previous_price) * (current_re - current_rfr)
        today_profits_highrfr = (current_position * previous_price) * (current_re - current_high_rfr)
        
        current_margin += today_profits
        current_margin_high += today_profits_highrfr
        
    if current_position < 0:
        current_re = (previous_price - current_price) / previous_price
        
        today_profits = (current_position * previous_price) * (current_re + current_rfr)
        today_profits_highrfr = (current_position * previous_price) * (current_re + current_high_rfr)
        
        current_margin += today_profits
        current_margin_high += today_profits_highrfr
    
    if current_margin < 0:
        print(f"Game Over! You lost all your money.")
        return cash, margin, position, re, bank_account, price, high_bank_account
    
    current_position = 0
    
    current_cash = current_bank_account
    current_cash += current_margin
    
    current_cash_high = currnet_high_bank_account
    current_cash_high += current_margin_high
    
    cash.append(current_cash)
    cash_high.append(current_cash_high)
    
    margin.append(current_margin)
    margin_high.append(current_margin_high)
    print(f"Final cash: {current_cash}")
    print(f"Final cash high: {current_cash_high}")
    print(f"Final margin: {current_margin}")
    print(f"Final margin high: {current_margin_high}")
    
    position.append(current_position)
    re.append(current_re)
    bank_account.append(current_bank_account)
    high_bank_account.append(currnet_high_bank_account)
    
    return cash,cash_high, margin, margin_high, position, re, bank_account, high_bank_account
    
cash, cash_high,  margin, margin_high, position, re, bank_account, high_bank_account = first_trading_strategy(dataset)

--------------------------------------------------------------
new count 0
current_position: -58207.21600095414, current_price: 34.360001, current_margin: 200000
current_position*current_price:2000000.0; current_margin*leverage_factor: 2000000
difference: 0.0
--------------------------------------------------------------
new count 1
current_position: -58207.21600095414, current_price: 34.630001, current_margin: 215603.2499075594
current_position*current_price:2015715.948320258; current_margin*leverage_factor: 2156032.499075594
difference: -140316.55075533595
--------------------------------------------------------------
new count 2
current_position: -58207.21600095414, current_price: 34.639999, current_margin: 216070.82177194275
current_position*current_price:2016297.9040658355; current_margin*leverage_factor: 2160708.2177194273
difference: -144410.3136535918
--------------------------------------------------------------
new count 3
current_position: 60536.38388011897, current_price: 3

In [5]:
report_df = pd.DataFrame({
    'margin': margin[1:],
    "margin_high_rfr": margin_high[1:],
    'position': position[1:],
    'cash': cash[1:],
    "cash_with_high_rfr": cash_high[1:],
    "bank_account": bank_account[1:],
    "bank_account_with_high_rfr": high_bank_account[1:],
    'excess_return': re[1:],
})
report_df.index = dataset.index

In [6]:
target_data = dataset.copy()
target_data = target_data.drop(columns=['Open', 'High', 'Low', 'Volume', 'Adj Close', "excess return", "return", "Tomorrow Prediction"])
final_result = pd.concat([target_data, report_df], axis=1)

In [7]:
final_result["theta"] = final_result["position"] * final_result["Close"]

final_result["pnl"] = final_result["margin"] - final_result["margin"].shift(1)
final_result["pnl_capital"] = final_result["bank_account"] - final_result["bank_account"].shift(1)
final_result["pnl_total"] = final_result["cash"] - final_result["cash"].shift(1)

final_result["cumulative_pnl"] = final_result["pnl"].cumsum()
final_result["cumulative_pnl_capital"] = final_result["pnl_capital"].cumsum()
final_result["cumulative_pnl_total"] = final_result["pnl_total"].cumsum()

final_result["turn_over_theta"] = abs(final_result["theta"] - final_result["theta"].shift(1))
final_result["turn_over_theta"] = final_result["turn_over_theta"].fillna(0)
final_result["cumulative_turn_over_theta"] = final_result["turn_over_theta"].cumsum()
final_result["turn_over_units"] = abs(final_result["position"] - final_result["position"].shift(1))
final_result["turn_over_units"] = final_result["turn_over_units"].fillna(0)
final_result["Upper_bound"] = final_result["margin"] * leverage_factor
final_result["Lower_bound"] = -final_result["margin"] * leverage_factor

final_result["pnl_150"] = final_result["margin_high_rfr"] - final_result["margin_high_rfr"].shift(1)
final_result["pnl_capital_150"] = final_result["bank_account_with_high_rfr"] - final_result["bank_account_with_high_rfr"].shift(1)
final_result["pnl_total_150"] = final_result["cash_with_high_rfr"] - final_result["cash_with_high_rfr"].shift(1)

final_result["cumulative_pnl_150"] = final_result["pnl_150"].cumsum()
final_result["cumulative_pnl_capital_150"] = final_result["pnl_capital_150"].cumsum()
final_result["cumulative_pnl_total_150"] = final_result["pnl_total_150"].cumsum()
final_result.fillna(0, inplace=True)

In [8]:
# final_result.to_csv("./validation/ARIMA_validation.csv")
final_result.to_csv("./testing/ARIMA_testing.csv")